# State of Data 2021 - Análise

Análise da pesquisa do State of Data 2021, feito pelo Data Hackers em conjunto com a Consultoria Bain.

In [1]:
#import numpy as np
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import seaborn as sns
import re

## Ler dados

In [2]:
df0 = pd.read_csv(
    r'..\data\raw\State of Data 2021 - Dataset - Pgina1.csv', 
    low_memory = False
)

In [3]:
df0.head()

,"('P0', 'id')","('P1_a ', 'Idade')","('P1_a_a ', 'Faixa idade')","('P1_b ', 'Genero')","('P1_e ', 'Estado onde mora')","('P1_e_a ', 'uf onde mora')","('P1_e_b ', 'Regiao onde mora')","('P1_g_b ', 'Regiao de origem')","('P1_g_c ', 'Mudou de Estado?')","('P1_h ', 'Nivel de Ensino')",...,"('P8_d_g ', 'Criando e dando manutenção em ETLs, DAGs e automações de pipelines de dados.')","('P8_d_h ', 'Criando e gerenciando soluções de Feature Store e cultura de MLOps.')","('P8_d_i ', 'Criando e mantendo a infra que meus modelos e soluções rodam (clusters, servidores, API, containers, etc.)')","('P9_a ', 'Quais das iniciativas do Data Hackers que você já acessou/acompanhou?')","('P9_a_a ', 'Blog/Medium do Data Hackers')","('P9_a_b ', 'Podcast do Data Hackers')","('P9_a_d ', 'Newsletter Semanal')","('P9_a_e ', 'Canal do Slack')","('P9_a_f ', 'Canal do Youtube do Data Hackers')","('P9_a_g ', 'Ainda não conhecia o Data Hackers')"
0,qkx4q0ei90wcjxnqkx4q0j3xgf0zn13s,38.0,35-39,Masculino,Ceará (CE),CE,Nordeste,NaN,0,Pós-graduação,...,NaN,NaN,NaN,"Newsletter Semanal, Podcast do Data Hackers",0,1,1,0,0,0
1,zdl2n19yhgpnoaco6kkczdl2nwv9zwrt,39.0,35-39,Masculino,Bahia (BA),BA,Nordeste,Sudeste,1,Pós-graduação,...,NaN,NaN,NaN,"Newsletter Semanal, Blog/Medium do Data Hackers",1,0,1,0,0,0
2,vsamqp2un3q7us84mgvsams5fulsmcoh,30.0,30-34,Masculino,Santa Catarina (SC),SC,Sul,NaN,0,Pós-graduação,...,NaN,NaN,NaN,"Blog/Medium do Data Hackers, Podcast do Data H...",1,1,0,0,0,0
3,v31ab41botodnsv31zgg4k34zzojy81l,38.0,35-39,Feminino,São Paulo (SP),SP,Sudeste,NaN,0,Pós-graduação,...,NaN,NaN,NaN,Ainda não conhecia o Data Hackers,0,0,0,0,0,1
4,ubhu8ntvm4xc0sfkdubhu80e973eek0k,36.0,35-39,Masculino,Santa Catarina (SC),SC,Sul,NaN,0,Pós-graduação,...,NaN,NaN,NaN,"Podcast do Data Hackers, Canal do Slack, Newsl...",0,1,1,1,0,0


In [4]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2645 entries, 0 to 2644
Columns: 356 entries, ('P0', 'id') to ('P9_a_g ', 'Ainda não conhecia o Data Hackers')
dtypes: float64(293), int64(7), object(56)
memory usage: 7.2+ MB


### Transformar títulos das colunas

Os títulos das colunas serão só os códigos das perguntas; a pergunta completa estará na `series` `pergunta_key`.

In [6]:
pat = r'\([\'\"](.+)\s*[\'\"]\s*\,\s*[\'\"](.+)\s*[\'\"]\)'

pergunta_desc0 = [ re.findall(pat, col)[0] for col in df0.columns ]

Transformar colunas mal-codificadas.

In [7]:
pergunta_desc = []
for id, q in pergunta_desc0:
    id = str.strip(id)
    q = str.strip(q)
    
    if id.endswith('_'):
        id += q[0]
        q = q[2:]
    
    pergunta_desc.append((id, q))


In [8]:
df0.columns = [ str.strip(p[0]) for p in pergunta_desc ]
pergunta_key = pd.Series(
    [ p[1] for p in pergunta_desc ],
    index = df0.columns
)

In [9]:
df0.head()

,P0,P1_a,P1_a_a,P1_b,P1_e,P1_e_a,P1_e_b,P1_g_b,P1_g_c,P1_h,...,P8_d_g,P8_d_h,P8_d_i,P9_a,P9_a_a,P9_a_b,P9_a_d,P9_a_e,P9_a_f,P9_a_g
0,qkx4q0ei90wcjxnqkx4q0j3xgf0zn13s,38.0,35-39,Masculino,Ceará (CE),CE,Nordeste,NaN,0,Pós-graduação,...,NaN,NaN,NaN,"Newsletter Semanal, Podcast do Data Hackers",0,1,1,0,0,0
1,zdl2n19yhgpnoaco6kkczdl2nwv9zwrt,39.0,35-39,Masculino,Bahia (BA),BA,Nordeste,Sudeste,1,Pós-graduação,...,NaN,NaN,NaN,"Newsletter Semanal, Blog/Medium do Data Hackers",1,0,1,0,0,0
2,vsamqp2un3q7us84mgvsams5fulsmcoh,30.0,30-34,Masculino,Santa Catarina (SC),SC,Sul,NaN,0,Pós-graduação,...,NaN,NaN,NaN,"Blog/Medium do Data Hackers, Podcast do Data H...",1,1,0,0,0,0
3,v31ab41botodnsv31zgg4k34zzojy81l,38.0,35-39,Feminino,São Paulo (SP),SP,Sudeste,NaN,0,Pós-graduação,...,NaN,NaN,NaN,Ainda não conhecia o Data Hackers,0,0,0,0,0,1
4,ubhu8ntvm4xc0sfkdubhu80e973eek0k,36.0,35-39,Masculino,Santa Catarina (SC),SC,Sul,NaN,0,Pós-graduação,...,NaN,NaN,NaN,"Podcast do Data Hackers, Canal do Slack, Newsl...",0,1,1,1,0,0


### Alterar títulos
#### Página 1

In [10]:
pergunta_key[pergunta_key.index.str.startswith('P1')]

P1_a                 Idade
P1_a_a         Faixa idade
P1_b                Genero
P1_e      Estado onde mora
P1_e_a        uf onde mora
P1_e_b    Regiao onde mora
P1_g_b    Regiao de origem
P1_g_c    Mudou de Estado?
P1_h       Nivel de Ensino
P1_i      Área de Formação
dtype: object

In [11]:
def alterar_tipos_p1(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    df.loc[df['P1_a'].isna(), 'P1_a'] = pd.NA
    df['P1_a'] = df['P1_a'].astype(pd.Int8Dtype())
    
    df['P1_a_a'] = df['P1_a_a'].astype('category')
    df['P1_b'] = df['P1_b'].astype('category')
    df['P1_e'] = df['P1_e'].astype('category')
    df['P1_e_a'] = df['P1_e_a'].astype('category')
    df['P1_e_b'] = df['P1_e_b'].astype('category')
    df['P1_g_b'] = df['P1_g_b'].astype('category')
    df['P1_g_c'] = df['P1_g_c'].astype('bool')
    df['P1_h'] = df['P1_h'].astype('category')
    df['P1_i'] = df['P1_i'].astype('category')

    return df

In [12]:
(df0
    .pipe(alterar_tipos_p1)
).head()

,P0,P1_a,P1_a_a,P1_b,P1_e,P1_e_a,P1_e_b,P1_g_b,P1_g_c,P1_h,...,P8_d_g,P8_d_h,P8_d_i,P9_a,P9_a_a,P9_a_b,P9_a_d,P9_a_e,P9_a_f,P9_a_g
0,qkx4q0ei90wcjxnqkx4q0j3xgf0zn13s,38,35-39,Masculino,Ceará (CE),CE,Nordeste,NaN,False,Pós-graduação,...,NaN,NaN,NaN,"Newsletter Semanal, Podcast do Data Hackers",0,1,1,0,0,0
1,zdl2n19yhgpnoaco6kkczdl2nwv9zwrt,39,35-39,Masculino,Bahia (BA),BA,Nordeste,Sudeste,True,Pós-graduação,...,NaN,NaN,NaN,"Newsletter Semanal, Blog/Medium do Data Hackers",1,0,1,0,0,0
2,vsamqp2un3q7us84mgvsams5fulsmcoh,30,30-34,Masculino,Santa Catarina (SC),SC,Sul,NaN,False,Pós-graduação,...,NaN,NaN,NaN,"Blog/Medium do Data Hackers, Podcast do Data H...",1,1,0,0,0,0
3,v31ab41botodnsv31zgg4k34zzojy81l,38,35-39,Feminino,São Paulo (SP),SP,Sudeste,NaN,False,Pós-graduação,...,NaN,NaN,NaN,Ainda não conhecia o Data Hackers,0,0,0,0,0,1
4,ubhu8ntvm4xc0sfkdubhu80e973eek0k,36,35-39,Masculino,Santa Catarina (SC),SC,Sul,NaN,False,Pós-graduação,...,NaN,NaN,NaN,"Podcast do Data Hackers, Canal do Slack, Newsl...",0,1,1,1,0,0


In [13]:
df0[[col for col in df0.columns if col.startswith('P1')]].isna().sum()

P1_a        28
P1_a_a       0
P1_b         0
P1_e        36
P1_e_a      36
P1_e_b       0
P1_g_b    2161
P1_g_c       0
P1_h         0
P1_i        64
dtype: int64

#### Página 2

In [14]:
pergunta_key[pergunta_key.index.str.startswith('P2')]

P2_a                   Qual sua situação atual de trabalho?
P2_b                                                  Setor
P2_c                                 Numero de Funcionarios
P2_d                                                Gestor?
P2_e                                      Cargo como Gestor
P2_f                                            Cargo Atual
P2_g                                                  Nivel
P2_h                                         Faixa salarial
P2_i      Quanto tempo de experiência na área de dados v...
P2_j      Quanto tempo de experiência na área de TI/Enge...
P2_k             Você está satisfeito na sua empresa atual?
P2_l      Qual o principal motivo da sua insatisfação co...
P2_l_a    Falta de oportunidade de crescimento no empreg...
P2_l_b             Salário atual não corresponde ao mercado
P2_l_c       Não tenho uma boa relação com meu líder/gestor
P2_l_d    Gostaria de trabalhar em em outra área de atuação
P2_l_e                  Gostaria de rece

In [15]:
df0[[col for col in df0.columns if col.startswith('P2')]].isna().sum()

P2_a         0
P2_b       280
P2_c       280
P2_d       280
P2_e      2137
P2_f       788
P2_g       788
P2_h       280
P2_i       280
P2_j       280
P2_k       283
P2_l      2033
P2_l_a    2033
P2_l_b    2033
P2_l_c    2033
P2_l_d    2033
P2_l_e    2033
P2_l_f    2033
P2_l_g    2033
P2_m       283
P2_n       280
P2_o       294
P2_o_a     285
P2_o_b     285
P2_o_c     285
P2_o_d     285
P2_o_e     285
P2_o_f     285
P2_o_g     285
P2_o_h     285
P2_o_i     285
P2_o_j     285
P2_q       280
P2_r       280
P2_s       280
dtype: int64

In [16]:
df0[[col for col in df0.columns if col.startswith('P2')]].head().transpose()

,0,1,2,3,4
P2_a,Empregado (CLT),Empreendedor ou Empregado (CNPJ),Empregado (CLT),Empreendedor ou Empregado (CNPJ),Empreendedor ou Empregado (CNPJ)
P2_b,Marketing,Consultoria,Indústria,Serviços,Varejo
P2_c,de 101 a 500,de 1 a 5,de 1001 a 3000,de 1001 a 3000,de 6 a 10
P2_d,1.0,1.0,1.0,1.0,1.0
P2_e,Supervisor/Coordenador,"Sócio ou C-level (CEO, CDO, CIO, CTO etc)",Supervisor/Coordenador,Gerente/Head/Diretor/VP,Gerente/Head/Diretor/VP
P2_f,NaN,NaN,NaN,NaN,NaN
P2_g,NaN,NaN,NaN,NaN,NaN
P2_h,de R$ 4.001/mês a R$ 6.000/mês,de R$ 6.001/mês a R$ 8.000/mês,de R$ 8.001/mês a R$ 12.000/mês,de R$ 12.001/mês a R$ 16.000/mês,de R$ 8.001/mês a R$ 12.000/mês
P2_i,Mais de 10 anos,de 2 a 3 anos,de 1 a 2 anos,Mais de 10 anos,de 1 a 2 anos
P2_j,Não tive experiência na área de TI/Engenharia ...,Não tive experiência na área de TI/Engenharia ...,de 2 a 3 anos,Menos de 1 ano,de 4 a 5 anos


In [17]:
(df0['P2_b'] == 'Outro').sum()

122

In [18]:
def alterar_tipos_p2(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df['P2_a'] = df['P2_a'].astype('category')

    df.loc[df['P2_b'].isna(), 'P2_b'] = 'Outro'
    df['P2_b'] = df['P2_b'].astype('category')
    
    df['P2_c'] = df['P2_c'].astype('category')
    df['P2_d'] = df['P2_d'].astype('bool')
    df['P2_e'] = df['P2_e'].astype('category')
    df['P2_f'] = df['P2_f'].astype('category')
    df['P2_g'] = df['P2_g'].astype(CategoricalDtype(
        categories = ('Júnior', 'Pleno', 'Sênior'),
        ordered = True
    ))
   
    # range de salário
    p2h_interval_dict = {
        'Menos de R$ 1.000/mês': pd.Interval(0, 1000, closed='both'),
        'de R$ 1.001/mês a R$ 2.000/mês': pd.Interval(1000, 2000, closed='right'),
        'de R$ 2.001/mês a R$ 3000/mês': pd.Interval(2000, 3000, closed='right'),
        'de R$ 3.001/mês a R$ 4.000/mês': pd.Interval(3000, 4000, closed='right'),
        'de R$ 4.001/mês a R$ 6.000/mês': pd.Interval(4000, 6000, closed='right'),
        'de R$ 6.001/mês a R$ 8.000/mês': pd.Interval(6000, 8000, closed='right'),
        'de R$ 8.001/mês a R$ 12.000/mês': pd.Interval(8000, 12000, closed='right'),
        'de R$ 12.001/mês a R$ 16.000/mês': pd.Interval(12000, 16000, closed='right'),
        'de R$ 16.001/mês a R$ 20.000/mês': pd.Interval(16000, 20000, closed='right'),
        'de R$ 20.001/mês a R$ 25.000/mês': pd.Interval(20000, 25000, closed='right'),
        'de R$ 25.001/mês a R$ 30.000/mês': pd.Interval(25000, 30000, closed='right'),
        'de R$ 30.001/mês a R$ 40.000/mês': pd.Interval(30000, 40000, closed='right'),
        'Acima de R$ 40.001/mês': pd.Interval(4000, np.inf, closed='neither'),
    }

    # range de salário em string
    df['P2_h'] = df['P2_h'].astype(CategoricalDtype(
        categories = p2h_interval_dict.keys(),
        ordered = True
    ))

    # range de salário em intervalo do pandas (pd.Interval)
    df['P2_h_interval'] = df['P2_h'].map(p2h_interval_dict).astype(CategoricalDtype(categories = p2h_interval_dict.values(), ordered = True))

    # experiencia da área de dados
    p2i_interval_dict = {
        'Não tenho experiência na área de dados': pd.Interval(-np.inf, 0, closed='right'),
        'Menos de 1 ano': pd.Interval(0, 1, closed='right'),
        'de 1 a 2 anos': pd.Interval(1, 2, closed='right'),
        'de 2 a 3 anos': pd.Interval(2, 3, closed='right'),
        'de 4 a 5 anos': pd.Interval(4, 5, closed='right'),
        'de 6 a 10 anos': pd.Interval(6, 10, closed='right'),
        'Mais de 10 anos': pd.Interval(10, np.inf, closed='neither'),
    }

    # experiencia na área de dados em string
    df['P2_i'] = df['P2_i'].astype(CategoricalDtype(
        categories = p2i_interval_dict.keys(),
        ordered = True
    ))

    # experiência na área de dados em intervalo do pandas (pd.Interval)
    df['P2_i_interval'] = df['P2_i'].map(p2i_interval_dict).astype(CategoricalDtype(categories = p2i_interval_dict.values(), ordered = True))

    # experiencia da área de ti antes de entrar na área de dados
    p2j_interval_dict = {
        'Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados': pd.Interval(-np.inf, 0, closed='right'),
        'Menos de 1 ano': pd.Interval(0, 1, closed='right'),
        'de 1 a 2 anos': pd.Interval(1, 2, closed='right'),
        'de 2 a 3 anos': pd.Interval(2, 3, closed='right'),
        'de 4 a 5 anos': pd.Interval(4, 5, closed='right'),
        'de 6 a 10 anos': pd.Interval(6, 10, closed='right'),
        'Mais de 10 anos': pd.Interval(10, np.inf, closed='neither'),
    }

    # experiencia na área de dados em string
    df['P2_j'] = df['P2_j'].astype(CategoricalDtype(
        categories = p2j_interval_dict.keys(),
        ordered = True
    ))

    # experiência na área de dados em intervalo do pandas (pd.Interval)
    df['P2_j_interval'] = df['P2_j'].map(p2j_interval_dict).astype(CategoricalDtype(categories = p2j_interval_dict.values(), ordered = True))
    df['P2_k'] = df['P2_k'].astype('bool')
    
    # subopcoes dentro de 'motivo de insatisfacao com o emprego atual'
    for letra in 'abcdefg':
        df[f'P2_l_{letra}'] = df[f'P2_l_{letra}'].astype('bool')

    # subopcoes dentro de 'criterios para decidir onde trabalhar'
    for letra in 'abcdefghij':
        df[f'P2_o_{letra}'] = df[f'P2_o_{letra}'].astype('bool')
    
    df['P2_q'] = df['P2_q'].astype('category')
    df['P2_r'] = df['P2_r'].astype('category')
    df['P2_s'] = df['P2_s'].astype('category')
    
    return df

In [19]:
(df0
    .pipe(alterar_tipos_p1)
    .pipe(alterar_tipos_p2)
).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2645 entries, 0 to 2644
Columns: 359 entries, P0 to P2_j_interval
dtypes: Int8(1), bool(20), category(23), float64(273), int64(6), object(36)
memory usage: 6.5+ MB


#### Página 3

In [20]:
pergunta_key[pergunta_key.index.str.startswith('P3')]

P3_a      Qual o número aproximado de pessoas que atuam ...
P3_b      Quais desses papéis/cargos fazem parte do time...
P3_b_a                                   Analytics Engineer
P3_b_b                    Engenharia de Dados/Data Engineer
P3_b_c                       Analista de Dados/Data Analyst
P3_b_d                    Cientista de Dados/Data Scientist
P3_b_e                           Database Administrator/DBA
P3_b_f                 Analista de Business Intelligence/BI
P3_b_g                    Arquiteto de Dados/Data Architect
P3_b_h                             Data Product Manager/DPM
P3_b_i                                     Business Analyst
P3_c      Quais dessas responsabilidades fazem parte da ...
P3_c_a    Pensar na visão de longo prazo de dados da emp...
P3_c_b    Organização de treinamentos e iniciativas com ...
P3_c_c    Atração, seleção e contratação de talentos par...
P3_c_d    Decisão sobre contratação de ferramentas e tec...
P3_c_e    Sou gestor da equipe responsáv

In [21]:
df0[[col for col in df0.columns if col.startswith('P3')]].isna().sum()

P3_a      2137
P3_b      2166
P3_b_a    2163
P3_b_b    2163
P3_b_c    2163
P3_b_d    2163
P3_b_e    2163
P3_b_f    2163
P3_b_g    2163
P3_b_h    2163
P3_b_i    2163
P3_c      2138
P3_c_a    2138
P3_c_b    2138
P3_c_c    2138
P3_c_d    2138
P3_c_e    2138
P3_c_f    2138
P3_c_g    2138
P3_c_h    2138
P3_c_i    2138
P3_c_j    2138
P3_c_k    2138
P3_d      2137
P3_d_a    2137
P3_d_b    2137
P3_d_c    2137
P3_d_d    2137
P3_d_e    2137
P3_d_f    2137
P3_d_g    2137
P3_d_h    2137
P3_d_i    2137
P3_d_j    2137
P3_d_k    2137
P3_d_l    2137
P3_d_m    2137
P3_d_n    2137
dtype: int64

In [22]:
df0[[col for col in df0.columns if col.startswith('P3')]].head().transpose()

,0,1,2,3,4
P3_a,4 - 10,1 - 3,Ainda não temos pessoas atuando com dados na e...,1 - 3,Ainda não temos pessoas atuando com dados na e...
P3_b,Analista de Business Intelligence/BI,Analista de Business Intelligence/BI,Analista de Business Intelligence/BI,Analista de Business Intelligence/BI,Analista de Business Intelligence/BI
P3_b_a,0.0,0.0,0.0,0.0,0.0
P3_b_b,0.0,0.0,0.0,0.0,0.0
P3_b_c,0.0,0.0,0.0,0.0,0.0
P3_b_d,0.0,0.0,0.0,0.0,0.0
P3_b_e,0.0,0.0,0.0,0.0,0.0
P3_b_f,1.0,1.0,1.0,1.0,1.0
P3_b_g,0.0,0.0,0.0,0.0,0.0
P3_b_h,0.0,0.0,0.0,0.0,0.0


In [30]:
def alterar_tipos_p3(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # experiencia da área de dados
    p3a_interval_dict = {
        'Ainda não temos pessoas atuando com dados na empresa': pd.Interval(-np.inf, 0, closed='right'),
        '1 - 3': pd.Interval(1, 3, closed='both'),
        '4 - 10': pd.Interval(4, 10, closed='both'),
        '11 - 20': pd.Interval(11, 20, closed='both'),
        '21 - 50': pd.Interval(21, 50, closed='both'),
        '51 - 100': pd.Interval(51, 100, closed='both'),
        '101 - 300': pd.Interval(101, 300, closed='both'),
        'Acima de 300 pessoas': pd.Interval(300, np.inf, closed='left'),
    }

    # experiencia na área de dados em string
    df['P3_a'] = df['P3_a'].astype(CategoricalDtype(
        categories = p3a_interval_dict.keys(),
        ordered = True
    ))

    # experiência na área de dados em intervalo do pandas (pd.Interval)
    df['P3_a_interval'] = df['P3_a'].map(p3a_interval_dict).astype(CategoricalDtype(categories = p3a_interval_dict.values(), ordered = True))

    df['P3_b'] = df['P3_b'].astype('category')
    # subopcoes dentro de 'papeis do time de dados da sua empresa'
    for letra in 'abcdefghi':
        df[f'P3_b_{letra}'] = df[f'P3_b_{letra}'].astype('bool')
    
    # subopcoes dentro de 'responsabilidades como gestor'
    for letra in 'abcdefghijk':
        df[f'P3_c_{letra}'] = df[f'P3_c_{letra}'].astype('bool')
    
    # subopcoes dentro de 'desafios como gestor'
    for letra in 'abcdefghijklmn':
        df[f'P3_d_{letra}'] = df[f'P3_d_{letra}'].astype('bool')
    
    return df

In [32]:
(df0
    .pipe(alterar_tipos_p1)
    .pipe(alterar_tipos_p2)
    .pipe(alterar_tipos_p3)
).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2645 entries, 0 to 2644
Columns: 360 entries, P0 to P3_a_interval
dtypes: Int8(1), bool(54), category(26), float64(239), int64(6), object(34)
memory usage: 5.9+ MB


In [25]:
df0['P3_d'].unique()

array(['Garantir a manutenção dos projetos e modelos em produção, em meio ao crescimento da empresa., Garantir retorno do investimento (ROI) em projetos de dados., Conseguir gerar valor para as áreas de negócios através de estudos e experimentos.',
       'Convencer a empresa a aumentar os investimentos na área de dados.',
       'Dividir o tempo entre entregas técnicas e gestão de pessoas., Conseguir levar inovação para a empresa através dos dados., Gestão de projetos envolvendo áreas multidisciplinares da empresa.',
       'Conseguir levar inovação para a empresa através dos dados., Organizar as informações e garantir a qualidade e confiabilidade., Convencer a empresa a aumentar os investimentos na área de dados.',
       'Garantir retorno do investimento (ROI) em projetos de dados., Gestão de projetos envolvendo áreas multidisciplinares da empresa., Convencer a empresa a aumentar os investimentos na área de dados.',
       'Dividir o tempo entre entregas técnicas e gestão de pessoas

In [29]:
pergunta_key['P3_d']

'Quais são os 3 maiores desafios que você tem como gestor no atual momento?'

In [27]:
pergunta_key[pergunta_key.index.str.startswith('P3')].index

Index(['P3_a', 'P3_b', 'P3_b_a', 'P3_b_b', 'P3_b_c', 'P3_b_d', 'P3_b_e',
       'P3_b_f', 'P3_b_g', 'P3_b_h', 'P3_b_i', 'P3_c', 'P3_c_a', 'P3_c_b',
       'P3_c_c', 'P3_c_d', 'P3_c_e', 'P3_c_f', 'P3_c_g', 'P3_c_h', 'P3_c_i',
       'P3_c_j', 'P3_c_k', 'P3_d', 'P3_d_a', 'P3_d_b', 'P3_d_c', 'P3_d_d',
       'P3_d_e', 'P3_d_f', 'P3_d_g', 'P3_d_h', 'P3_d_i', 'P3_d_j', 'P3_d_k',
       'P3_d_l', 'P3_d_m', 'P3_d_n'],
      dtype='object')